In [ ]:
#! jupyter nbconvert --to script test.ipynb

In [ ]:
from azureml.core import Workspace

# Provide the subscription ID, resource group name, and workspace name
subscription_id = "0dd49ea4-49ce-412e-b274-385ffb01c578"
resource_group = "MachineLearning-RG"
workspace_name = "OHMS-AzureML"

# Connect to your workspace
try:
    workspace = Workspace(subscription_id, resource_group, workspace_name)
    print("Workspace connection successful!")
    print("Workspace name:", workspace.name)
    print("Azure region:", workspace.location)
    print("Resource group:", workspace.resource_group)
except Exception as e:
    print("Workspace connection failed:", str(e))

In [ ]:
from azureml.core.authentication import InteractiveLoginAuthentication
from azureml.core import Workspace, Environment, Experiment, ScriptRunConfig
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.run import Run
import numpy as np
from sklearn.linear_model import LinearRegression
import joblib

In [ ]:
auth = InteractiveLoginAuthentication(tenant_id="864324a0-fbca-44b4-a025-ee7be7426f0b")
ws = Workspace.from_config(auth=auth)

In [ ]:
conda_dep = CondaDependencies().create(conda_packages=["scikit-learn==1.1"])
myenv = Environment(name="myenv2")
myenv.python.conda_dependencies = conda_dep

In [ ]:
celsius_q = np.array([-40, -10, 0, 8, 15, 22, 38], dtype=float)
fahrenheit_a = np.array([-40, 14, 32, 46, 59, 72, 100], dtype=float)

In [ ]:
X = celsius_q.reshape(-1, 1)
model = LinearRegression()
model.fit(X, fahrenheit_a)
joblib.dump(model, "model.pkl")

In [ ]:
experiment_name = "test-25"
experiment = Experiment(workspace=ws, name=experiment_name)

In [ ]:
script_config = ScriptRunConfig(
    source_directory=".",
    script="test.py",  # Update script name if necessary
    compute_target="VM-ML-2Cores-14GB",  # Update compute target if necessary
    environment=myenv,
)

In [11]:
run = experiment.submit(config=script_config)
run.wait_for_completion(show_output=True)